In [1]:
import numpy as np
import pandas as pd
import os
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from nltk.corpus import stopwords
import keras.preprocessing.text as T
from keras.preprocessing.text import Tokenizer

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GanJinZERO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
print("Train shape : ",train.shape)
print("Test shape : ",test.shape)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)


In [5]:
text_list = train["question_text"]

In [6]:
# make word_dict
word_dict = dict()
for i in tqdm(range(len(text_list))):
    text_seg = T.text_to_word_sequence(text_list.values[i])
    for j in text_seg:
        if j in word_dict:
            word_dict[j] += 1
        else:
            word_dict[j] = 1        

100%|█████████████████████████████████████████████████████████████████████| 1306122/1306122 [00:22<00:00, 59054.25it/s]


In [7]:
#print(word_dict)
print(len(word_dict))

222161


In [8]:
pop_list = []
for word in word_dict:
    if word_dict.get(word) < 200:
        pop_list += [word]
for word in pop_list:
    word_dict.pop(word)

In [9]:
#print(word_dict)
print(len(word_dict))

5894


In [10]:
# Pop stop words
list_stop_words = list(set(stopwords.words('english')))
for word in list_stop_words:
    if word in word_dict:
        word_dict.pop(word)
#print(word_dict)
print(len(word_dict))

5748


In [11]:
word_order = dict()
order = 0
for word in word_dict:
    word_order[word] = order
    order += 1

In [12]:
# use_text_length = len(text_list)
use_text_length = 20000

In [13]:
x_input = np.zeros((use_text_length, order))
for i in tqdm(range(use_text_length)):
    x_i = np.zeros((order))
    text_seg = T.text_to_word_sequence(text_list.values[i])
    for word in text_seg:
        if word in word_order:
            x_i[word_order[word]] += 1
    x_i = x_i.reshape(1, -1)
    x_input[i] = x_i
x_input_f = x_input.reshape(-1, len(word_order))

100%|█████████████████████████████████████████████████████████████████████████| 20000/20000 [00:01<00:00, 17803.08it/s]


In [14]:
x_input_f.shape

(20000, 5748)

In [15]:
y = train["target"]

In [16]:
y = y.values[0:use_text_length]

In [17]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_input_f, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [18]:
test

,qid,question_text
0,00014894849d00ba98a9,My voice range is A2-C5. My chest voice goes u...
1,000156468431f09b3cae,How much does a tutor earn in Bangalore?
2,000227734433360e1aae,What are the best made pocket knives under $20...
3,0005e06fbe3045bd2a92,Why would they add a hypothetical scenario tha...
4,00068a0f7f41f50fc399,What is the dresscode for Techmahindra freshers?
5,000a2d30e3ffd70c070d,How well are you adapting to the Trump era?
6,000b67672ec9622ff761,What should be the last thing people do in life?
7,000b7fb1146d712c1105,Received conditional offer for Masters in Inte...
8,000d665a8ddc426a1907,What does appareils photo mean in French?
9,000df6fd2229447b2969,Is there a system of Public Interest Litigatio...


In [19]:
test_list = test["question_text"]
test_length = len(test_list)
test_input = np.zeros((test_length, order))
for i in tqdm(range(test_length)):
    x_i = np.zeros((order))
    text_seg = T.text_to_word_sequence(test_list.values[i])
    for word in text_seg:
        if word in word_order:
            x_i[word_order[word]] += 1
    x_i = x_i.reshape(1, -1)
    test_input[i] = x_i
test_input_f = test_input.reshape(-1, len(word_order))

100%|█████████████████████████████████████████████████████████████████████████| 56370/56370 [00:03<00:00, 18280.82it/s]


In [20]:
y_pred = clf.predict(test_input_f)

In [21]:
sum(y_pred)/len(y_pred)

0.06375731772219266

In [22]:
out_df = pd.DataFrame({"qid":test["qid"].values})
out_df['prediction'] = y_pred
out_df.to_csv("submission.csv", index=False)